import packages
------------------

Some packages are installed automatically if you use Anaconda. As pytorch is used here, you are expected to install that in your machine. 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import sys
sys.path.append('/content/gdrive/MyDrive/UTRECHT/utils')
sys.path.insert(0,'/content/gdrive/MyDrive/UTRECHT')

import utils

In [ ]:
from __future__ import print_function, division
import os, random, time, copy
from skimage import io, transform, morphology, feature
import numpy as np
import os.path as path
import scipy.io as sio
from scipy import misc
from scipy import ndimage, signal
import scipy
import pickle
import sys
import math
import matplotlib.pyplot as plt
import PIL.Image
from io import BytesIO
from skimage import data, img_as_float
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from random import shuffle

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler 
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms

#sys.path.append('/home/skong2/project/dpff4ldl')
#from utils.metrics import *
#from utils.metrics import *
#from utils.flow_functions import *
#from models.pem import *
#from models.PEM_compact import *
#from fetchData.datasetCOMBINED import *
#from trainingProtocol.trainval_COM2s_intFlow_LazyPrior_bidirection import *
#from trainingProtocol.trainval_COM2s_intFlow_sLazyPrior_bidirectionAllLayers import *
#from losses import *
#from losses_suppl import *

from utils.flow_functions import *

import warnings # ignore warnings
warnings.filterwarnings("ignore")
print(sys.version)
print(torch.__version__)

3.8.10 (default, Nov 14 2022, 12:59:47) 
[GCC 9.4.0]
1.13.1+cu116


Setup config parameters
 -----------------
 
 There are several things to setup, like which GPU to use, where to read images and save files, etc. Please read and understand this. By default, you should be able to run this script smoothly by changing nothing.

In [ ]:
path_to_image = '/content/gdrive/MyDrive/UTRECHT/Detection/Training_Tiles3'
path_to_label = '/content/gdrive/MyDrive/UTRECHT/Detection/Masks10_smallGrains'
path_to_annotCombo = '/content/gdrive/MyDrive/UTRECHT/Detection/AnnotCombo10_smallGrains'

#if not os.path.exists(path_to_annotCombo): 
#    os.makedirs(path_to_annotCombo)

In [ ]:
#### set project name, save directory for the project
project_name = '02_Exp1_SmallGrains'
trials = ['Trial_01','Trial_02','Trial_03','Trial_04']
trial = trials[3]

exp_dir = '/content/gdrive/MyDrive/exp' # experiment directory, used for reading the init model
exp_dir = os.path.join(exp_dir, project_name)
save_dir = os.path.join(exp_dir, trial) 
if not os.path.exists(save_dir): 
    os.makedirs(save_dir)

In [ ]:
# make a list of all the image stacks

tmpSampleList = [sample for sample in os.listdir(path_to_label)
                 if os.path.isdir(os.path.join(path_to_label, sample))]

shuffle(tmpSampleList)
len(tmpSampleList)

211

In [ ]:
# get the training and test sets for the general detector
gen_det_save_dir = '/content/gdrive/MyDrive/exp/01_Generalized_Model_FullTrainingData/Trial_03'
with open(os.path.join(gen_det_save_dir,'dbinfo.pkl'), 'rb') as handle:
    dbinfo_all = pickle.load(handle)    

print(sorted(dbinfo_all['test_det_list']))

['PALIV_22-23_slide 1_scan4_101', 'PALIV_22-23_slide 1_scan4_102', 'PALIV_22-23_slide 1_scan4_103', 'PALIV_22-23_slide 1_scan4_104', 'PALIV_22-23_slide 1_scan4_105', 'PALIV_22-23_slide 1_scan4_106', 'PALIV_22-23_slide 1_scan4_107', 'PALIV_22-23_slide 1_scan4_108', 'PALIV_22-23_slide 1_scan4_110', 'PALIV_22-23_slide 1_scan4_112', 'PALIV_22-23_slide 1_scan4_113', 'PALIV_22-23_slide 1_scan4_114', 'PALIV_22-23_slide 1_scan4_118', 'PALIV_22-23_slide 1_scan4_121', 'PALIV_22-23_slide 1_scan4_123', 'PALIV_22-23_slide 1_scan4_129', 'PALIV_22-23_slide 1_scan4_132', 'PALIV_22-23_slide 1_scan4_133', 'PALIV_22-23_slide 1_scan4_134', 'PALIV_22-23_slide 1_scan4_135', 'PALIV_22-23_slide 1_scan4_137', 'PALIV_22-23_slide 1_scan4_138', 'PALIV_22-23_slide 1_scan4_139', 'PALIV_22-23_slide 1_scan4_140', 'PALIV_22-23_slide 1_scan4_143', 'PALIV_22-23_slide 1_scan4_146', 'PALIV_22-23_slide 1_scan4_150', 'PALIV_22-23_slide 1_scan4_158', 'PALIV_22-23_slide 1_scan4_162', 'PALIV_22-23_slide 1_scan4_163', 'PALIV_22

In [ ]:
# see which of the small grain images are in test-set-all. It looks like exactly 80%, 42 images, so we keep this subset. 
test_det_list = []
for i in tmpSampleList:
  if i in dbinfo_all['test_det_list']:
    test_det_list.append(i)

print(sorted(test_det_list))
print(len(test_det_list))

['PALIV_22-23_slide 1_scan4_106', 'PALIV_22-23_slide 1_scan4_108', 'PALIV_22-23_slide 1_scan4_133', 'PALIV_22-23_slide 1_scan4_138', 'PALIV_22-23_slide 1_scan4_208', 'PALIV_22-23_slide 1_scan4_255', 'PALIV_22-23_slide 1_scan4_284', 'PALIV_22-23_slide 1_scan4_98', 'PALIV_22-23_slide 1_scan4_99', 'PALIV_23-24_slide 1_scan1_116', 'PALIV_23-24_slide 1_scan1_59', 'PALIV_36-37_slide 1_scan1_100', 'PALIV_36-37_slide 1_scan1_109', 'PALIV_36-37_slide 1_scan1_181', 'PALIV_36-37_slide 1_scan1_213', 'PALIV_36-37_slide 1_scan1_35', 'PALIV_36-37_slide 1_scan1_372', 'PALIV_36-37_slide 1_scan1_385', 'PALIV_36-37_slide 1_scan1_42', 'PALIV_36-37_slide 1_scan1_65', 'PALIV_45-46_glycerine_slide 1_scan1_291', 'PALIV_45-46_glycerine_slide 1_scan1_365', 'PALIV_45-46_glycerine_slide 1_scan2_102', 'PALIV_45-46_glycerine_slide 1_scan2_130', 'PALIV_45-46_glycerine_slide 1_scan2_131', 'PALIV_45-46_glycerine_slide 1_scan3_10', 'PALIV_45-46_glycerine_slide 1_scan3_115', 'PALIV_45-46_glycerine_slide 1_scan3_311', 'P

In [ ]:
# same for the train set.

train_det_list = []
for i in tmpSampleList:
  if i in dbinfo_all['train_det_list']:
    train_det_list.append(i)

print(sorted(train_det_list))
print(len(train_det_list))

['PALIV_20-21_slide1_scan3_132', 'PALIV_20-21_slide1_scan3_167', 'PALIV_20-21_slide1_scan3_250', 'PALIV_20-21_slide1_scan3_330', 'PALIV_20-21_slide1_scan3_337', 'PALIV_20-21_slide1_scan3_53', 'PALIV_20-21_slide1_scan3_541', 'PALIV_20-21_slide1_scan3_577', 'PALIV_20-21_slide1_scan3_634', 'PALIV_20-21_slide1_scan3_652', 'PALIV_20-21_slide1_scan3_659', 'PALIV_20-21_slide1_scan3_71', 'PALIV_20-21_slide1_scan3_75', 'PALIV_20-21_slide1_scan3_88', 'PALIV_21-22_slide 1_scan1_246', 'PALIV_21-22_slide 1_scan1_334', 'PALIV_21-22_slide 1_scan1_90', 'PALIV_21-22_slide 1_scan2_165', 'PALIV_21-22_slide 1_scan2_182', 'PALIV_21-22_slide 1_scan2_227', 'PALIV_21-22_slide 1_scan2_348', 'PALIV_21-22_slide 1_scan2_357', 'PALIV_21-22_slide 1_scan2_377', 'PALIV_21-22_slide 1_scan2_9', 'PALIV_22-23_slide 1_scan2_129', 'PALIV_22-23_slide 1_scan2_135', 'PALIV_22-23_slide 1_scan2_200', 'PALIV_22-23_slide 1_scan2_279', 'PALIV_22-23_slide 1_scan2_314', 'PALIV_22-23_slide 1_scan2_329', 'PALIV_22-23_slide 1_scan2_364

In [ ]:
# # split the dataset into 80% training and 20% testing sets

# # train_det_dict = {}
# # test_det_dict = {}
# train_det_list = []
# test_det_list = []

# trRatio = 0.8
# if len(tmpSampleList)>0:
#     trNum = round(len(tmpSampleList)*trRatio)
#     tsNum = len(tmpSampleList) - trNum
    
#     train_det_list += tmpSampleList[:trNum]
#     test_det_list += tmpSampleList[trNum:]               
# else: 
#     train_det_list += tmpSampleList

In [ ]:
# save all the info in a pickle file to be read in the next notebook
dbinfo = {}
dbinfo['train_det_list'] = train_det_list
dbinfo['test_det_list'] = test_det_list

with open(os.path.join(save_dir,'dbinfo.plk'), 'wb') as handle:
    pickle.dump(dbinfo, handle) # , protocol=pickle.HIGHEST_PROTOCOL

In [ ]:
# see the lengths of the training and test datasets
with open(os.path.join(save_dir,'dbinfo.plk'), 'rb') as handle:
    dbinfo = pickle.load(handle)

dbinfo.keys(), len(dbinfo['train_det_list']), len(dbinfo['test_det_list'])

(dict_keys(['train_det_list', 'test_det_list']), 171, 40)

In [ ]:
dbinfo

{'train_det_list': ['PALIV_B_Sample_32-33_glyc_scan1_21',
  'PALIV_25-26_slide 1_scan2_94',
  'PALIV_25-26_slide 1_scan2_82',
  'PALIV_B_Sample_31-32_scan2_398',
  'PALIV_B_19_slide2_scan1_261',
  'PALIV_22-23_slide 1_scan2_96',
  'PALIV_B_17.8_slide1_scan2_404',
  'PALIV_B_Sample_26-27_glyc_scan1_178',
  'PALIV_B_17.8_slide1_scan2_52',
  'PALIV_22-23_slide 1_scan2_83',
  'PALIV_B_Sample_31-32_scan2_400',
  'PALIV_B_19_slide2_scan1_389',
  'PALIV_27-28_glycerine_scan 2_294',
  'PALIV_27-28_glycerine_scan 2_112',
  'PALIV_20-21_slide1_scan3_132',
  'PALIV_B_Sample_33-34_glyc_scan1_93',
  'PALIV_B_17.8_slide1_scan2_83',
  'PALIV_27-28_slide 1_scan1_130',
  'PALIV_B_Sample_32-33_glyc_scan1_153',
  'PALIV_B_Sample_31-32_scan3_313',
  'PALIV_22-23_slide 1_scan3_15',
  'PALIV_B_Sample_26-27_glyc_scan3_4',
  'PALIV_22-23_slide 1_scan2_129',
  'PALIV_B_19_slide2_scan1_380',
  'PALIV_25-26_slide 1_scan2_379',
  'PALIV_B_Sample_33-34_glyc_scan1_137',
  'PALIV_B_Sample_26-27_glyc_scan3_183',
  'P

In [ ]:
# count the number of each taxa in the train and test sets
set_name = 'test' #'test'
sampleList = dbinfo[set_name+'_det_list']
print(len(sampleList))

40


In [ ]:
import pandas as pd
codes_set = set()
codes_list = []
# remove = ['ind','375', 'fun', 't58','t22','dia','ttr','ped','t47','p16','glo','p17','gel','spr','p2','neu','u4','u16','u17','his','u30','u45','peo','azo','bot']
small_grain = ['u/m', 'cec', 'mel']

for idx in range(len(sampleList)):
    current_example = sampleList[idx]
    current_image_path= os.path.join(path_to_image, current_example)

    path_to_imdb = os.path.join(str(current_image_path), 'annot2.csv')
    imdb = pd.read_csv(path_to_imdb, dtype=object)

    class_label_full = pd.DataFrame(imdb, columns = ['X Coord of Bb','Y Coord of Bb','Bb Width','Bb Height','Code'])
    class_label_full= class_label_full.to_numpy()
    # print(class_label_full)

    for i in range(len(class_label_full)):
        code = class_label_full[i][4]
        if code in small_grain:
            # continue
          codes_set.add(code)
          codes_list.append(code)

print(codes_set)
print(codes_list)


{'mel', 'cec', 'u/m'}
['u/m', 'cec', 'u/m', 'u/m', 'u/m', 'mel', 'mel', 'u/m', 'u/m', 'u/m', 'u/m', 'u/m', 'u/m', 'u/m', 'u/m', 'u/m', 'u/m', 'u/m', 'u/m', 'u/m', 'u/m', 'mel', 'u/m', 'u/m', 'u/m', 'u/m', 'u/m', 'u/m', 'cec', 'cec', 'u/m', 'u/m', 'u/m', 'mel', 'cec', 'u/m', 'u/m', 'u/m', 'u/m', 'u/m']


In [ ]:
gt_counts = {}
unique_codes = list(codes_set)
for j in range(len(unique_codes)):
  code = unique_codes[j]
  count = sum(x == code for x in codes_list)
  gt_counts.update({code:count})

print(gt_counts)

{'mel': 4, 'cec': 4, 'u/m': 32}
